In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import COLLECTIONS as cln
from IPython.display import clear_output
import html5lib

import matplotlib.pyplot as plt
%matplotlib inline

cnx = create_engine('postgresql://%s:%s@localhost:%s/%s' % (cln.username, 
                                                            cln.password, 
                                                            cln.port,
                                                            cln.db_name))

In [2]:
#Scrape Primary Airport Names
df_primary_airports = pd.read_html('https://en.wikipedia.org/wiki/List_of_airports_in_the_United_States', 
                                 attrs={"class":"wikitable"}, header=0)[0]
df_primary_airports.dropna(inplace=True)
df_primary_airports['Enplanements'] = df_primary_airports['Enplanements'].apply(lambda x: int(x.replace(',','').split(' ')[0]))
df_primary_airports.head()

,City,FAA,IATA,ICAO,Airport,Role,Enplanements
1,Birmingham,BHM,BHM,KBHM,Birmingham–Shuttlesworth International Airport,P-S,1325897
2,Dothan,DHN,DHN,KDHN,Dothan Regional Airport,P-N,46792
3,Huntsville,HSV,HSV,KHSV,Huntsville International Airport (Carl T. Jone...,P-S,519785
4,Mobile,MOB,MOB,KMOB,Mobile Regional Airport,P-N,278053
5,Montgomery,MGM,MGM,KMGM,Montgomery Regional Airport (Dannelly Field),P-N,175619


In [3]:
df_primary_airports = df_primary_airports[df_primary_airports['Enplanements'] > 500000]
df_primary_airports.to_sql('primary_airports', cnx, if_exists='replace')

In [4]:
make_table = {
    'airlines': False,
    'airports': False,
    'ontimeperformance': True,
    'weather': False,
    'weatherairportlinks': False,
    'weatherstations': False
}

In [5]:
# Update Airlines Table
table_name = 'airlines'

if make_table[table_name]:
    # Load Data
    df = pd.read_csv('airlines.dat', header=None)
    df.columns = ['AirlineID', 'Name', '', 'IATA', '', 'Callsign', 'Country', '']

    # Filter Data
    mask = ((df['AirlineID'] > 0) &
            (~df['IATA'].apply(str).isin(['-', 'nan'])) &
            (~df['Callsign'].apply(str).isin(['-', 'nan'])) &
            (df['Country'] == 'United States'))
    df.drop('', axis=1, inplace=True)
    df = df[mask]
    df.dropna()
    df['AirlineID'] = df['AirlineID'].apply(int)

    # Insert into table
    df.to_sql(table_name, cnx, if_exists='replace', index=False)

    

In [6]:
# Check Airlines Table
df_read = pd.read_sql_query("""SELECT * FROM Airlines""", cnx)
df_read.head()

,AirlineID,Name,IATA,Callsign,Country
0,10,40-Mile Air,Q5,MILE-AIR,United States
1,22,Aloha Airlines,AQ,ALOHA,United States
2,24,American Airlines,AA,AMERICAN,United States
3,35,Allegiant Air,G4,ALLEGIANT,United States
4,49,Airborne Express,GB,ABEX,United States


In [7]:
# Update Airlines Table
table_name = 'airports'

if make_table[table_name]:

    # Load Data
    df = pd.read_csv('airports.dat', header=None)
    df.columns = ['AirportID', 'Name', 'City', 'Country', 'IATA', '',
                  'Latitude', 'Longitude', 'Altitude', 'Timezone',
                  'DST', 'Tz_Database_Timezone', 'Type', 'Source']
    df.drop('', axis=1, inplace=True)

    df.dropna(inplace=True)
    df = df[df['Timezone'] != '\\N']

    df['AirportID'] = df['AirportID'].apply(int)
    df['Latitude'] = df['Latitude'].apply(float)
    df['Longitude'] = df['Longitude'].apply(float)
    df['Altitude'] = df['Altitude'].apply(int)
    df['Timezone'] = df['Timezone'].apply(float)

    df.to_sql(name=table_name, con=cnx, if_exists='replace', index=False)

In [8]:
# Check Airlines Table
df_read = pd.read_sql_query("""SELECT * FROM airports""", cnx)
df_read.head()

,AirportID,Name,City,Country,IATA,Latitude,Longitude,Altitude,Timezone,DST,Tz_Database_Timezone,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,-6.081690,145.391998,5282,10.0,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,-5.207080,145.789001,20,10.0,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,-5.826790,144.296005,5388,10.0,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,-6.569803,146.725977,239,10.0,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,-9.443380,147.220001,146,10.0,U,Pacific/Port_Moresby,airport,OurAirports


In [9]:
# Import WeatherStations
table_name = 'weatherstations'

if make_table[table_name]:
    with open('ghcnd-stations.txt', 'r') as file:
        x = file.read()
    x = x.split('\n')
    x = [i.strip() for i in x]
    x_left = [i.split()[:4] for i in x]
    x_right = [i[41:].split('  ')[0] for i in x]
    x_right = [i.strip() for i in x_right]
    x = [i + [j] for i,j in zip(x_left, x_right)]

    df = pd.DataFrame(x, columns=['StationID', 'Latitude', 'Longitude', 'Altitude_ft', 'Name'])
    df = df[df['StationID'] != '']
    df['Altitude_ft'] = df['Altitude_ft'].apply(float)*3.28084
    df['Name'] = df['Name'].apply(lambda item: ' '.join([i.capitalize() for i in item.split()]))
    df['Latitude'] = df['Latitude'].apply(float)
    df['Longitude'] = df['Longitude'].apply(float)

    df.to_sql(name=table_name, con=cnx, if_exists='replace', index=False)

In [10]:
# Check Airlines Table
df_read = pd.read_sql_query("""SELECT * FROM weatherstations""", cnx)
df_read.head()

,StationID,Latitude,Longitude,Altitude_ft,Name
0,ACW00011604,17.1167,-61.7833,33.136484,St Johns Coolidge Fld
1,ACW00011647,17.1333,-61.7833,62.992128,St Johns
2,AE000041196,25.3330,55.5170,111.548560,Sharjah Inter. Airp
3,AEM00041194,25.2550,55.3640,34.120736,Dubai Intl
4,AEM00041217,24.4330,54.6510,87.926512,Abu Dhabi Intl


In [11]:
# Create Weather Station Links
table_name = 'weatherairportlinks'

if make_table[table_name]:
    df_airports = pd.read_sql_query("""SELECT "IATA", "Latitude", "Longitude" FROM airports""", cnx)
    df_airports.set_index('IATA', inplace=True)
    df_stations = pd.read_sql_query("""SELECT "StationID", "Latitude", "Longitude", "Name" FROM weatherstations""", cnx)
    df_stations.set_index('StationID', inplace=True)

    df_link = {}
    i = 1
    for use_airport in df_airports.index.values:
        clear_output(wait=True)
        print('%d/%d' % (i, len(df_airports.index.values)))
    #     print(df_stations['Latitude'])
        if len(use_airport) != 3:
            i += 1
            continue
    #     print(use_airport)
    #     print(df_airports.loc[use_airport]['Latitude'])
        df_stations['lat_dif'] = abs(df_stations['Latitude'] - df_airports.loc[use_airport]['Latitude'])
        df_stations['lon_dif'] = abs(df_stations['Longitude'] - df_airports.loc[use_airport]['Longitude'])
        df_stations['sum_dif'] = df_stations['lon_dif'] + df_stations['lat_dif']
    #     print(df_stations.sort_values('sum_dif')['sum_dif'].iloc[0])
        df_link[use_airport] = df_stations.sort_values('sum_dif').index[0]
        i += 1

    df = pd.DataFrame(pd.Series(df_link)).reset_index()
    df.columns = ['IATA', 'StationID']

    df.to_sql(name=table_name, con=cnx, if_exists='replace', index=False)

In [12]:
# Import Weather Data
import shutil
import os
import zipfile
import gzip

table_name = 'weather'

if make_table[table_name]:
    path = 'Data/Weather/'
    idx = 1
    
    print("Make initial queries...")
    df_links = pd.read_sql_query("""SELECT * FROM weatherairportlinks""", cnx)
    df_completed_years = pd.read_sql_query("""select distinct extract(year from "Date") from weather;""", cnx)

    for fn in os.listdir(path):
        clear_output(wait=True)
        print("(%d/%d) %s" % (idx, len(os.listdir(path)), fn))
        print('Data')
        if not fn.endswith('.csv.gz'):
            continue
        if fn.split('.')[0] in [str(int(i[0])) for i in df_completed_years.values]:
            idx += 1
            continue

        print('Unzipping...')
        try:
            os.mkdir(path + 'output/', )
        except OSError:
            pass
        with gzip.open(path + fn, 'rb') as f_in, open(path + 'output/' + fn.replace('.gz',''), 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

        print('Reading CSV')
        df = pd.read_csv(path + 'output/' + fn.replace('.gz', ''), header=None)
        shutil.rmtree(path + 'output', )

        print('Processing Data')
        df.drop([4,5,6,7], axis=1, inplace=True)
        df.columns = ['StationID', 'Date', 'Obsv_Type', 'Obsv_Value']
        df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')
        df = df.merge(df_links, on='StationID')[['StationID', 'Date', 'Obsv_Type', 'Obsv_Value']]

        print('Adding to Database')
        sections = range(0, len(df), 1000000)
        for sect in range(len(sections)):
            if idx == 1 and sect == 0:
                if_exists = 'replace'
            else:
                if_exists = 'append'
            if sect == 0:
                subset_min = sections[sect]
                subset_max = sections[sect+1]
            elif sect == len(sections)-1:
                subset_min = sections[sect]
                subset_max = ''
            else:
                subset_min = sections[sect]
                subset_max = sections[sect+1]
            print('Part %d/%d (%s:%s)' % (sect+1, 
                                          len(sections),
                                          str(subset_min),
                                          str(subset_max)))
            if subset_max == '':
                df.iloc[subset_min:].to_sql(name=table_name, 
                                            con=cnx, 
                                            if_exists=if_exists, 
                                            index=False)
            else:
                df.iloc[subset_min:subset_max].to_sql(name=table_name, 
                                                      con=cnx, 
                                                      if_exists=if_exists, 
                                                      index=False)
        idx += 1

In [14]:
# Import Ontime Flight Performance Data
import shutil
import os
import zipfile

table_name = 'ontimeperformance'

if make_table[table_name]:
    path = 'Data/'

    def create_ids(data):
        ids = (data['FlightDate'].apply(lambda x: str(x).replace('-','_')) + 
               data['AirlineID'].apply(str) + 
               data['ArrTime'].apply(lambda x: str(x).split('.')[0]) + 
               data['DestAirportID'].apply(str) + 
               data['OriginAirportID'].apply(str) + 
               data['TailNum'].apply(str))
        return ids

    def int_to_time(x):
        import datetime as dt
        try:
            hour = int(x)//100
            if hour > 23:
                hour = hour - 24
            minute = int(x) - int(x)//100*100
            return dt.time(hour, minute)
        except:
            return dt.time(0, 0)

    i = 1
    current_files = pd.read_sql_query("""select distinct "File_Name" from ontimeperformance;""", cnx).values
    for fn in sorted(os.listdir(path)):
        if fn.startswith('On_Time_On_Time_Performance'):
            if fn in [i[0] for i in current_files]:
                print("Skipping File %s (already in database)" % fn)
                i += 1
                continue
            print('(%d/%d) %s' % (i, len(os.listdir(path)), fn))
            zip = zipfile.ZipFile(path+fn)  
            zip.extractall(path + 'output')
            df = pd.read_csv(path + 'output/' + fn.replace('.zip', '.csv'))
            shutil.rmtree(path + 'output', )
            base_row_count = len(df)
            df.dropna(subset=['ArrDelay'], inplace=True)
            df['PerformanceID'] = create_ids(df)
            df['File_Name'] = [fn]*len(df)
            df['FlightDate'] = pd.to_datetime(df['FlightDate'])
            print("Dropped %d Rows (%0.4f%%)." % (base_row_count - len(df), (base_row_count-len(df))/base_row_count*100))
            print('-----')
            print("Processing Data")
            keep_columns = ['PerformanceID', 'File_Name', 'FlightDate', 'Carrier', 'TailNum', 'FlightNum', 
                            'Origin', 'OriginCityName', 'OriginStateName',
                            'Dest', 'DestCityName', 'DestStateName',
                            'CRSDepTime', 'DepTime', 'DepTimeBlk', 'DepDelay', 'DepDelayMinutes', 'DepDel15',
                            'CRSArrTime', 'ArrTime', 'ArrTimeBlk', 'ArrDelay', 'ArrDelayMinutes', 'ArrDel15',
                            'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn',
                            'Cancelled', 'Diverted', 'CRSElapsedTime', 'ActualElapsedTime', 
                            'AirTime', 'Flights', 'Distance', 'DistanceGroup', 'CarrierDelay', 
                            'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 
                            'FirstDepTime', 'TotalAddGTime', 'LongestAddGTime']
            # Cleaning
            df['OriginCityName'] = df['OriginCityName'].apply(lambda x: x.split(',')[0])
            df['DestCityName'] = df['DestCityName'].apply(lambda x: x.split(',')[0])
            df['CRSDepTime'] = df['CRSDepTime'].apply(int_to_time)
            df['DepTime'] = df['DepTime'].apply(int_to_time)
            df['DepDelay'] = df['DepDelay'].apply(float)
            df['DepDelayMinutes'] = df['DepDelayMinutes'].apply(float)
            df['DepDel15'] = df['DepDel15'].apply(float)
            df['CRSArrTime'] = df['CRSArrTime'].apply(int_to_time)
            df['ArrTime'] = df['ArrTime'].apply(int_to_time)
            df['ArrDelay'] = df['ArrDelay'].apply(float)
            df['ArrDelayMinutes'] = df['ArrDelayMinutes'].apply(float)
            df['ArrDel15'] = df['ArrDel15'].apply(float)
            df['TaxiOut'] = df['TaxiOut'].apply(float)
            df['TaxiIn'] = df['TaxiIn'].apply(float)
            df['WheelsOff'] = df['WheelsOff'].apply(int_to_time)
            df['WheelsOn'] = df['WheelsOn'].apply(int_to_time)
            df['Cancelled'] = df['Cancelled'].apply(bool)
            df['Diverted'] = df['Diverted'].apply(bool)
            df['CRSElapsedTime'] = df['CRSElapsedTime'].apply(float)
            df['ActualElapsedTime'] = df['ActualElapsedTime'].apply(float)
            df['AirTime'] = df['AirTime'].apply(float)
            df['Flights'] = df['Flights'].apply(float)
            df['Distance'] = df['Distance'].apply(float)
            df['DistanceGroup'] = df['DistanceGroup'].apply(float)
            print("Writing to Database")
            if i == 1:
                if_exists = 'replace'
            else:
                if_exists = 'append'
            df[keep_columns].to_sql(name=table_name, 
                                    con=cnx, 
                                    if_exists=if_exists,
                                    index=False)
            i += 1

Skipping File On_Time_On_Time_Performance_2010_1.zip (already in database)
Skipping File On_Time_On_Time_Performance_2010_10.zip (already in database)
Skipping File On_Time_On_Time_Performance_2010_11.zip (already in database)
Skipping File On_Time_On_Time_Performance_2010_12.zip (already in database)
Skipping File On_Time_On_Time_Performance_2010_2.zip (already in database)
Skipping File On_Time_On_Time_Performance_2010_3.zip (already in database)
Skipping File On_Time_On_Time_Performance_2010_4.zip (already in database)
Skipping File On_Time_On_Time_Performance_2010_5.zip (already in database)
Skipping File On_Time_On_Time_Performance_2010_6.zip (already in database)
Skipping File On_Time_On_Time_Performance_2010_7.zip (already in database)
Skipping File On_Time_On_Time_Performance_2010_8.zip (already in database)
Skipping File On_Time_On_Time_Performance_2010_9.zip (already in database)
Skipping File On_Time_On_Time_Performance_2011_1.zip (already in database)
Skipping File On_Time_